### Домашняя работа к уроку 8
### Студент: Абрамов А.В.

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара 
1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)
4. сделать выводы, что получилось лучше

In [10]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [11]:
data = pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [12]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [13]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [15]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [16]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [17]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [18]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

### CNN

In [20]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 8s 296ms/step - loss: 1.3059 - accuracy: 0.6760 - val_loss: 1.1079 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 7s 286ms/step - loss: 0.8778 - accuracy: 0.7105 - val_loss: 0.8639 - val_accuracy: 0.7567
Epoch 3/5
23/23 [==============================] - 7s 291ms/step - loss: 0.7201 - accuracy: 0.7600 - val_loss: 0.7697 - val_accuracy: 0.7615
Epoch 4/5
23/23 [==============================] - 7s 293ms/step - loss: 0.6528 - accuracy: 0.7757 - val_loss: 0.7199 - val_accuracy: 0.7573
Epoch 5/5
23/23 [==============================] - 7s 292ms/step - loss: 0.6044 - accuracy: 0.7953 - val_loss: 0.7460 - val_accuracy: 0.7511


In [27]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])

metrics_df.loc[len(metrics_df.index)] = ['CNN', score[0], score[1]]

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 43ms/step - loss: 0.7839 - accuracy: 0.7318
Test score: 0.7838907241821289
Test accuracy: 0.731848955154419


### Simple RNN

In [28]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 8s 285ms/step - loss: 1.3180 - accuracy: 0.6605 - val_loss: 0.9223 - val_accuracy: 0.7176
Epoch 2/5
23/23 [==============================] - 6s 275ms/step - loss: 0.7960 - accuracy: 0.7526 - val_loss: 0.7243 - val_accuracy: 0.7698
Epoch 3/5
23/23 [==============================] - 7s 309ms/step - loss: 0.6523 - accuracy: 0.7775 - val_loss: 0.6854 - val_accuracy: 0.7781
Epoch 4/5
23/23 [==============================] - 8s 342ms/step - loss: 0.5829 - accuracy: 0.7956 - val_loss: 0.6703 - val_accuracy: 0.7836
Epoch 5/5
23/23 [==============================] - 9s 398ms/step - loss: 0.5232 - accuracy: 0.8158 - val_loss: 0.6801 - val_accuracy: 0.7812


In [29]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df.loc[len(metrics_df.index)] = ['Simple RNN', score[0], score[1]]

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 73ms/step - loss: 0.7057 - accuracy: 0.7633
Test score: 0.7057226300239563
Test accuracy: 0.7633107304573059


### CNN -> Simple RNN

In [31]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 9s 361ms/step - loss: 1.0840 - accuracy: 0.6841 - val_loss: 0.8600 - val_accuracy: 0.7228
Epoch 2/5
23/23 [==============================] - 8s 342ms/step - loss: 0.8439 - accuracy: 0.7210 - val_loss: 0.7509 - val_accuracy: 0.7535
Epoch 3/5
23/23 [==============================] - 8s 342ms/step - loss: 0.7314 - accuracy: 0.7598 - val_loss: 0.6944 - val_accuracy: 0.7660
Epoch 4/5
23/23 [==============================] - 8s 348ms/step - loss: 0.6515 - accuracy: 0.7763 - val_loss: 0.6629 - val_accuracy: 0.7746
Epoch 5/5
23/23 [==============================] - 8s 349ms/step - loss: 0.5926 - accuracy: 0.7905 - val_loss: 0.6563 - val_accuracy: 0.7784


In [32]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df.loc[len(metrics_df.index)] = ['CNN -> Simple RNN', score[0], score[1]]

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 66ms/step - loss: 0.6865 - accuracy: 0.7619
Test score: 0.6865180730819702
Test accuracy: 0.7618586421012878


### SimpleRNN -> CNN

In [33]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 13s 514ms/step - loss: 1.2470 - accuracy: 0.6864 - val_loss: 0.8799 - val_accuracy: 0.7197
Epoch 2/5
23/23 [==============================] - 13s 580ms/step - loss: 0.8306 - accuracy: 0.7315 - val_loss: 0.7393 - val_accuracy: 0.7601
Epoch 3/5
23/23 [==============================] - 16s 683ms/step - loss: 0.6677 - accuracy: 0.7705 - val_loss: 0.6780 - val_accuracy: 0.7660
Epoch 4/5
23/23 [==============================] - 16s 694ms/step - loss: 0.5653 - accuracy: 0.7961 - val_loss: 0.7277 - val_accuracy: 0.7705
Epoch 5/5
23/23 [==============================] - 17s 740ms/step - loss: 0.4860 - accuracy: 0.8247 - val_loss: 0.7904 - val_accuracy: 0.7605


In [34]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df.loc[len(metrics_df.index)] = ['Simple RNN -> CNN', score[0], score[1]]

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 113ms/step - loss: 0.8262 - accuracy: 0.7399
Test score: 0.8261706233024597
Test accuracy: 0.7399160861968994


In [35]:
metrics_df

,model,Test score,Test accuracy
0,CNN,0.783891,0.731849
1,Simple RNN,0.705723,0.763311
2,CNN -> Simple RNN,0.686518,0.761859
3,Simple RNN -> CNN,0.826171,0.739916


#### Вывод: наилучший результат показала Simple RNN